In [1]:
import cv2
import matplotlib.pyplot as plt
from os.path import isfile
import torch.nn.init as init
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import os
print(os.listdir("../input"))
from torch.utils.data import Dataset
from torchvision import transforms
from torch.optim import Adam, SGD, RMSprop
import time
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
import pickle
import cv2
import torch.nn.functional as F
from torchvision import models
import seaborn as sns
import random
import sys

['b4-regression-13', 'aptos2019-blindness-detection', 'efficientnet', 'b5-softmax-2', 'efficientnet-pytorch', 'b4-softmax-7', 'b5-regression-2', 'best-submission-v2']


In [2]:
from albumentations import (
    RandomBrightnessContrast,
    Compose, RandomSizedCrop,RandomSunFlare,Cutout
)

In [3]:
package_path = '../input/efficientnet/efficientnet-pytorch/EfficientNet-PyTorch/'
sys.path.append(package_path)

In [4]:
from efficientnet_pytorch import EfficientNet

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [6]:
num_classes = 5
seed_everything(1234)
IMG_SIZE    = 328
TTA         = 10

In [7]:
test = '../input/aptos2019-blindness-detection/test_images/'

In [8]:
def expand_path(p):
    p = str(p)
    if isfile(test + p + ".png"):
        return test + (p + ".png")
    return p

def p_show(imgs, label_name=None, per_row=3):
    n = len(imgs)
    rows = (n + per_row - 1)//per_row
    cols = min(per_row, n)
    fig, axes = plt.subplots(rows,cols, figsize=(15,15))
    for ax in axes.flatten(): ax.axis('off')
    for i,(p, ax) in enumerate(zip(imgs, axes.flatten())): 
        img = Image.open(expand_path(p))
        ax.imshow(img)
        ax.set_title(train_df[train_df.id_code == p].diagnosis.values)

In [9]:
def resize_images(img, desired_size = 1024):
    try:
        img = cv2.copyMakeBorder(img,10,10,10,10,cv2.BORDER_CONSTANT,value=[0,0,0])
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret,gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    
        contours,hierarchy = cv2.findContours(gray,
                                              cv2.RETR_EXTERNAL,
                                              cv2.CHAIN_APPROX_SIMPLE)
        contours = max(contours, key=cv2.contourArea)
        x,y,w,h = cv2.boundingRect(contours)

        if w>200 and h>200:
            new_img = img[y:y+h,x:x+w]
            height, width, _= new_img.shape

            if max([height, width]) > desired_size:
                ratio = float(desired_size/max([height, width]))
                new_img = cv2.resize(new_img, 
                                 tuple([int(width*ratio), int(height*ratio)]), 
                                 interpolation = cv2.INTER_CUBIC)
            return new_img
        else:
            return img
    except:
        return img

## Classification (b5 0.825)

In [10]:
class MyDataset(Dataset):
    
    def __init__(self, dataframe, argument=None, transform=None):
        self.df = dataframe
        self.transform = transform
        self.argument  = argument

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        p = self.df.id_code.values[idx]
        p_path = expand_path(p)
        image = cv2.imread(p_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        
        if self.argument:
            augmented = self.argument(image=image)
            image = augmented['image']
            image = transforms.ToPILImage()(image.astype(np.uint8))
        else:
            image = transforms.ToPILImage()(image)

        
        if self.transform:
            image = self.transform(image)
        
        return image #, label

In [11]:
test_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(360),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

testset      = MyDataset(pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv'), transform=test_transform)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=4)

In [12]:
model = EfficientNet.from_name('efficientnet-b5')
in_features = model._fc.in_features
model._fc = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_features, num_classes))
model.load_state_dict(torch.load('../input/b5-softmax-2/weight_acc_best.pt'))
model.cuda()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [13]:
for param in model.parameters():
    param.requires_grad = False

In [14]:
sample = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [15]:
%%time
test_pred_cls = np.zeros((len(sample)))

model.eval()
for _ in range(TTA):
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images = data
            images = images.cuda()
            pred   = model(images)
            pred   = torch.softmax(pred, dim=-1)
            pred   = np.argmax((pred.detach().cpu().numpy()), axis=1)
            test_pred_cls[i * 64:(i + 1) * 64] += pred #pred.detach().cpu().numpy()

31it [00:51,  1.18it/s]
31it [00:49,  1.16it/s]
31it [00:49,  1.20it/s]
31it [00:49,  1.13it/s]
31it [00:50,  1.25it/s]
31it [00:49,  1.18it/s]
31it [00:48,  1.25it/s]
31it [00:49,  1.60s/it]
31it [00:49,  1.16it/s]
31it [00:48,  1.23it/s]

CPU times: user 33.8 s, sys: 51.8 s, total: 1min 25s
Wall time: 8min 20s


In [16]:
output_cls = test_pred_cls / TTA
output_cls

array([2. , 3.1, 2.1, ..., 2. , 2.7, 3. ])

## Regression(b5 0.825)

In [17]:
%%time
seed_everything(1234)
IMG_SIZE = 328

class MyDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        label = self.df.diagnosis.values[idx]
        label = np.expand_dims(label, -1)
        
        p = self.df.id_code.values[idx]
        p_path = expand_path(p)
        image = cv2.imread(p_path)
        image = resize_images(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

test_transform = transforms.Compose([
    transforms.RandomAffine(degrees=360),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

testset      = MyDataset(pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv'), transform=test_transform)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=4)


model       = EfficientNet.from_name('efficientnet-b5')
in_features = model._fc.in_features
model._fc   = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_features, 1))
model.load_state_dict(torch.load('../input/b5-regression-2/weight_loss_best.pt'))
model.cuda()

for param in model.parameters():
    param.requires_grad = False

sample = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')


test_pred_reg = np.zeros((len(sample), 1))
model.eval()

for _ in range(TTA):
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images, _ = data
            images = images.cuda()
            pred = model(images)
            test_pred_reg[i * 64:(i + 1) * 64] += pred.detach().cpu().squeeze().numpy().reshape(-1, 1)

31it [01:02,  2.02s/it]
31it [01:01,  1.02it/s]
31it [01:01,  1.98s/it]
31it [01:01,  1.00it/s]
31it [01:01,  1.97s/it]
31it [01:02,  2.01s/it]
31it [01:01,  1.97s/it]
31it [01:01,  1.99s/it]
31it [01:01,  1.98s/it]
31it [01:01,  1.03s/it]

CPU times: user 34.3 s, sys: 48.3 s, total: 1min 22s
Wall time: 10min 19s


In [18]:
output_reg1 = test_pred_reg / TTA 
output_reg1

array([[1.85230896],
       [2.83469596],
       [2.31543038],
       ...,
       [2.30187526],
       [2.34037421],
       [2.11208034]])

## Regression (b4 0.821)

In [19]:
%%time
seed_everything(1234)
IMG_SIZE    = 256



class MyDataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        label = self.df.diagnosis.values[idx]
        label = np.expand_dims(label, -1)
        
        p = self.df.id_code.values[idx]
        p_path = expand_path(p)
        image = cv2.imread(p_path)
        image = resize_images(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

test_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.05, contrast=0.05),
    transforms.RandomAffine(degrees=360),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

testset      = MyDataset(pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv'), 
                         transform=test_transform)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=64, 
                                           shuffle=False, num_workers=4)

model       = EfficientNet.from_name('efficientnet-b4')
in_features = model._fc.in_features
model._fc   = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_features, 1))
model.load_state_dict(torch.load('../input/b4-regression-13/weight_loss_best(1).pt'))
model.cuda()

for param in model.parameters():
    param.requires_grad = False

sample        = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_pred_reg = np.zeros((len(sample), 1))
model.eval()

for _ in range(TTA):
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images, _ = data
            images = images.cuda()
            pred = model(images)
            test_pred_reg[i * 64:(i + 1) * 64] += pred.detach().cpu().squeeze().numpy().reshape(-1, 1)

31it [00:59,  1.91s/it]
31it [00:59,  1.92s/it]
31it [00:59,  1.92s/it]
31it [00:59,  1.91s/it]
31it [00:58,  1.90s/it]
31it [00:59,  1.93s/it]
31it [00:59,  1.91s/it]
31it [00:58,  1.90s/it]
31it [00:59,  1.92s/it]
31it [00:59,  1.91s/it]

CPU times: user 23 s, sys: 20.1 s, total: 43.1 s
Wall time: 9min 56s


In [20]:
output_reg2 = test_pred_reg / TTA 
output_reg2

array([[1.79122491],
       [2.7146647 ],
       [2.37552838],
       ...,
       [2.37801898],
       [2.84338799],
       [2.19399108]])

## Classification (b4:0.820)

In [21]:
%%time
seed_everything(1234)
IMG_SIZE    = 256

class MyDataset(Dataset):
    
    def __init__(self, dataframe, argument=None, transform=None):
        self.df = dataframe
        self.transform = transform
        self.argument  = argument

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        
        p = self.df.id_code.values[idx]
        p_path = expand_path(p)
        image = cv2.imread(p_path)
        image = resize_images(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        
        if self.argument:
            augmented = self.argument(image=image)
            image = augmented['image']
            image = transforms.ToPILImage()(image.astype(np.uint8))
        else:
            image = transforms.ToPILImage()(image)

        if self.transform:
            image = self.transform(image)
        
        return image

test_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.05, contrast=0.05),
    transforms.RandomAffine(degrees=360),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

testset      = MyDataset(pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv'), 
                         transform=test_transform)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=4)

model = EfficientNet.from_name('efficientnet-b4')
in_features = model._fc.in_features
model._fc = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_features, num_classes))
model.load_state_dict(torch.load('../input/b4-softmax-7/weight_acc_best (1).pt'))
model.cuda()

for param in model.parameters():
    param.requires_grad = False

sample = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
test_pred = np.zeros((len(sample)))

model.eval()
for _ in range(TTA):
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader)):
            images = data
            images = images.cuda()
            pred   = model(images)
            pred   = torch.softmax(pred, dim=-1)
            pred   = np.argmax((pred.detach().cpu().numpy()), axis=1)
            test_pred[i * 128:(i + 1) * 128] += pred 

16it [01:00,  3.77s/it]
16it [00:59,  3.71s/it]
16it [00:59,  3.72s/it]
16it [00:59,  3.70s/it]
16it [00:59,  3.70s/it]
16it [00:59,  3.73s/it]
16it [00:59,  3.70s/it]
16it [00:58,  3.68s/it]
16it [00:59,  3.69s/it]
16it [00:59,  3.70s/it]

CPU times: user 17 s, sys: 27.4 s, total: 44.4 s
Wall time: 9min 57s


In [22]:
output_cls2 = test_pred / TTA 
output_cls2

array([2. , 2.8, 2.2, ..., 2. , 2. , 2. ])

## Blend

In [23]:
output = ((output_cls*0.5 + output_reg1.flatten()*0.5)*0.5 + output_reg2.flatten()*0.5)*0.8 + output_cls2*0.2

In [24]:
output

array([1.88695176, 2.83280507, 2.27329743, ..., 2.21158264, 2.54543004,
       2.3000125 ])

In [25]:
coef = [0.57, 1.37, 2.57, 3.57]

In [26]:
for i, pred in enumerate(output):
    if pred < coef[0]:
        output[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        output[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        output[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        output[i] = 3
    else:
        output[i] = 4

submission = pd.DataFrame({'id_code':pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv').id_code.values,
                          'diagnosis':output.astype(np.int32)})

print(submission.head())
submission.to_csv('submission.csv', index=False)
print(os.listdir('./'))

        id_code  diagnosis
0  0005cfc8afb6          2
1  003f0afdcd15          3
2  006efc72b638          2
3  00836aaacf06          2
4  009245722fa4          2


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

['__output__.json', 'submission.csv', '__notebook__.ipynb']


/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [27]:
#best_sub = pd.read_csv('../input/best-submission-v2/submission.csv')
#from sklearn.metrics import mean_squared_error, cohen_kappa_score
#print(cohen_kappa_score(submission.diagnosis.values[:len(best_sub)], best_sub.diagnosis.values, weights="quadratic"))
#print(mean_squared_error(submission.diagnosis.values[:len(best_sub)], best_sub.diagnosis.values))